In [11]:
import pandas as pd
import os

"""
Como ira organizado el DataFrame

Games | ReleaseDate | CharactersPath | Developer | 2D | 3D | TagTeam | AirDashers | Anime | WeaponBased | Fast-paced | Footsies | SlowPaced | Installbased | CrossOver
Nombre | Fecha de Lanzamiento | Ruta_en_dir_data (del archivo con los links de personajes)|ImageLink | Desarrollador | y el resto con una X mayuscula si tiene esa caracteristica o un espacio si no la tiene
"""


df = pd.DataFrame({"Games":[],
                   "ReleaseDate":[],
                   "CharactersPath":[],
                   "ImageLink":[], #Link de la imagen del juego 
                   "2D":[], 
                   "3D":[], 
                   "TagTeam":[], 
                   "AirDashers":[], 
                   "Anime":[], 
                   "WeaponBased":[], 
                   "Fast-paced":[],
                   "Footsies":[],
                   "SlowPaced":[],
                   "Installbased":[],
                   "CrossOver":[]
                   })



#funcion que crea un archivo csv con el nombre del juego y la link de discord de la imagen de los personajes.

def CreateCharacterFile(GameName)-> None:
    file_name =  f"Data/{GameName}.csv"

    if os.path.exists(file_name):
        
        return None

    with open(file_name, "w") as f:

        # First row is the header, put CharacterName and ImageLink
        f.write("CharacterName,ImageLink,Archetype,Archetype2,Archetype3,Archetype4,Archetype5\n")
        f.close()



In [14]:

"""
Agregar un nuevo juego por medio de los archivos tabulados
Los archivos tabulados deben de estar en la carpeta TabulatedData
"""

with open(f"TabulatedData/GameName.csv", "r") as f:
    rows_to_add = []
    for line in f:
            # Desempaquetar y eliminar espacios en blanco
            GameName, ReleaseDate = map(str.strip, line.split(","))
            
            # Diccionario para la nueva fila
            new_row = {
                "Games": GameName,
                "ReleaseDate": ReleaseDate,
                "CharactersPath": f"Data/{GameName}.csv",
                "ImageLink": "",
                "2D": "",
                "3D": "",
                "TagTeam": "",
                "AirDashers": "",
                "Anime": "",
                "WeaponBased": "",
                "Fast-paced": "",
                "Footsies": "",
                "SlowPaced": "",
                "Installbased": "",
                "CrossOver": ""
            }
            
            # Añadir a la lista de filas a agregar
            rows_to_add.append(new_row)

    # Usar pandas.concat para agregar todas las filas al DataFrame
    df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)

In [ ]:

"""
Volver el dataFrame del pirmer bloque en un CSV
"""

df.to_csv("Games.csv", index=False) 

In [ ]:
import hashlib
import time

def hasher_function(To_Hash):
    hasher = hashlib.sha256()
    
    for hash in To_Hash:
        hasher.update(To_Hash.encode('utf-8'))
    

    current_time = str(int(time.time()))  
    hasher.update(current_time.encode('utf-8'))
    

    hash_value = hasher.hexdigest()
    
    return "U" + hash_value


In [19]:
import pandas as pd
import neo4j as neo
import os
from dotenv import load_dotenv

load_dotenv()

# Driver for graph
driver = neo.GraphDatabase.driver(os.getenv('NEO4J_URI'), auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD')))

# Leer el CSV
csv_path = "Games.csv"
data = pd.read_csv(csv_path)

usuarios_csv_path = "Respuestas.csv"
usuarios_data = pd.read_csv(usuarios_csv_path)

# Crear nodos y relaciones en Neo4j
with driver.session() as session:
    # Crear nodos para juegos
    for index, row in data.iterrows():
        game_name = row['Games']
        release_date = row['ReleaseDate']
        characters_path = row['CharactersPath']
        image_link = row['ImageLink']

        session.run("CREATE (g:Reconode {name: $name, image_link: $image_link})",
                    name=game_name, image_link=image_link)

        characters_path = f"Data/{game_name}.csv"
        # Commented cause there
        # for index,row in characters_path.iterrows():
        #     character_name = row['CharacterName']
        #     image_link = row['ImageLink']
        #     archetypes = [row['Archetype'], row['Archetype2'], row['Archetype3'], row['Archetype4'], row['Archetype5']]

        #     session.run("CREATE (c:Character {name: $name, image_link: $image_link, archetypes: [$archetype, $archetype2, $archetype3, $archetype4, $archetype5]})",
        #                 name=character_name, image_link=image_link, archetype=archetypes[0], archetype2=archetypes[1], archetype3=archetypes[2], archetype4=archetypes[3], archetype5=archetypes[4])

        #     session.run("MATCH (g:Reconode {name: $game_name}), (c:Character {name: $character_name}) CREATE (c)-[:CHARACTER_FROM]->(g)",
        #                 game_name=game_name, character_name=character_name)         
        
        #Para cada categoría, si hay una 'X', crear el nodo de categoría y la relación
        category_columns = ["2D", "3D", "TagTeam", "AirDashers", "Anime", "WeaponBased", "Fast-paced", 
                            "Footsies", "SlowPaced", "Installbased", "CrossOver"]

        for category in category_columns:
            if pd.notna(row[category]) and (row[category] == 'X' or row[category] == 'x'):
                print(category)
                session.run("MERGE (c:Label {name: $category})", category=category)

                session.run("MATCH (g:Reconode {name: $game_name}), (c:Label {name: $category}) CREATE (g)-[:BELONGS_TO]->(c)",
                            game_name=game_name, category=category)
        

driver.close()



3D
SlowPaced
3D
SlowPaced
3D
SlowPaced
3D
Fast-paced
Footsies
3D
Fast-paced
Footsies
3D
Fast-paced
Footsies
3D
Fast-paced
Footsies
3D
Fast-paced
3D
Fast-paced
Footsies
3D
Fast-paced
Footsies
3D
Fast-paced
3D
Footsies
3D
Footsies
3D
Fast-paced
Footsies
3D
Footsies
3D
Footsies
2D
AirDashers
Anime
2D
AirDashers
Anime
Fast-paced
2D
AirDashers
Anime
Fast-paced
2D
AirDashers
Anime
Fast-paced
2D
AirDashers
Anime
Fast-paced
Footsies
2D
TagTeam
AirDashers
Fast-paced
CrossOver
2D
TagTeam
AirDashers
Fast-paced
CrossOver
2D
SlowPaced
2D
SlowPaced
2D
Footsies
SlowPaced
2D
SlowPaced
2D
SlowPaced
2D
Footsies
SlowPaced
2D
Footsies
SlowPaced
2D
Footsies
2D
Footsies
2D
Footsies
2D
Fast-paced
Footsies
2D
Fast-paced
Footsies
2D
Fast-paced
Footsies
2D
Footsies
SlowPaced
2D
Footsies
SlowPaced
2D
Footsies
SlowPaced
2D
Footsies
SlowPaced
2D
Footsies
SlowPaced
2D
Footsies
2D
Footsies
2D
Footsies
2D
Footsies
2D
Footsies
2D
Fast-paced
Footsies
2D
Footsies


In [ ]:
#TODO: Fix this
with driver.session() as session:
        for index, row in usuarios_data.iterrows():
            # Obtener el nombre del usuario segun el CSV
            user_name = row['Usuarios']
            hashed_user_name = hasher_function(user_name)  # Hashear el nombre de usuario
            preferences = row.drop('Usuarios').to_dict()
            print(preferences)
            # Crear nodo de usuario con el nombre hasheado
            session.run("CREATE (u:User {name: $name})", name=hashed_user_name)
            
            # Vincular el usuario a los juegos según sus preferencias
            for game, value in preferences.items():
                if pd.notna(value) and (value == 'x'):
                    # Vincular el usuario al "Reconode" del juego
                    session.run("MATCH (u:User {name: $user_name}), (g:Reconode {name: $game_name}) CREATE (u)-[:LIKES]->(g)",
                                user_name=hashed_user_name, game_name=game)